In [1]:
import pandas as pd
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [2]:
print(len(documents))
print(documents[:5])

1103663
                                       headline_text  index
0  aba decides against community broadcasting lic...      0
1     act fire witnesses must be aware of defamation      1
2     a g calls for infrastructure protection summit      2
3           air nz staff in aust strike for pay rise      3
4      air nz strike to affect australian travellers      4


In [3]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [5]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kevin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kevin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
print(WordNetLemmatizer().lemmatize('went', pos='v'))

go


In [10]:
stemmer = SnowballStemmer('english')
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]
pd.DataFrame(data = {'original word': original_words, 'stemmed': singles})

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [11]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [12]:
doc_sample = documents[documents['index'] == 300].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample)

original document: 
['iran', 'military', 'plane', 'crash', 'kills', '302']


 tokenized and lemmatized document: 
['iran', 'militari', 'plane', 'crash', 'kill']


In [13]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

In [15]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [17]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [18]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [19]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4209]

[(27, 1), (137, 1), (3970, 1)]

In [20]:
bow_doc_4209 = bow_corpus[4209]
for i in range(len(bow_doc_4209)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4209[i][0], 
                                               dictionary[bow_doc_4209[i][0]], 
bow_doc_4209[i][1]))

Word 27 ("aussi") appears 1 time.
Word 137 ("expect") appears 1 time.
Word 3970 ("fleme") appears 1 time.


In [23]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)

In [24]:
corpus_tfidf = tfidf[bow_corpus]

In [25]:
from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5893154583024485),
 (1, 0.3892866165028569),
 (2, 0.49651921997736453),
 (3, 0.5046106271280878)]


In [26]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [27]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.022*"warn" + 0.020*"market" + 0.018*"hospit" + 0.016*"health" + 0.014*"share" + 0.014*"break" + 0.014*"minist" + 0.013*"school" + 0.013*"student" + 0.013*"worker"
Topic: 1 
Words: 0.026*"adelaid" + 0.022*"test" + 0.017*"open" + 0.017*"tasmanian" + 0.017*"win" + 0.014*"melbourn" + 0.013*"australian" + 0.012*"vote" + 0.011*"darwin" + 0.011*"parti"
Topic: 2 
Words: 0.024*"chang" + 0.019*"miss" + 0.017*"tasmania" + 0.014*"farmer" + 0.014*"concern" + 0.013*"fall" + 0.013*"busi" + 0.013*"show" + 0.012*"price" + 0.012*"search"
Topic: 3 
Words: 0.030*"govern" + 0.022*"south" + 0.021*"north" + 0.017*"countri" + 0.015*"rural" + 0.015*"australia" + 0.014*"hour" + 0.014*"west" + 0.013*"indigen" + 0.011*"communiti"
Topic: 4 
Words: 0.021*"die" + 0.019*"live" + 0.017*"donald" + 0.016*"coast" + 0.014*"gold" + 0.014*"lose" + 0.013*"abus" + 0.013*"return" + 0.012*"park" + 0.012*"royal"
Topic: 5 
Words: 0.027*"elect" + 0.022*"sydney" + 0.017*"say" + 0.014*"labor" + 0.014*"record" + 0.

In [28]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.011*"coast" + 0.010*"south" + 0.009*"sport" + 0.009*"north" + 0.007*"east" + 0.007*"marriag" + 0.007*"west" + 0.007*"wednesday" + 0.007*"march" + 0.006*"festiv"
Topic: 1 Word: 0.010*"donald" + 0.009*"royal" + 0.008*"commiss" + 0.008*"david" + 0.008*"crash" + 0.007*"hobart" + 0.007*"thursday" + 0.006*"energi" + 0.006*"truck" + 0.006*"scott"
Topic: 2 Word: 0.010*"elect" + 0.009*"abbott" + 0.008*"juli" + 0.007*"peter" + 0.007*"care" + 0.006*"toni" + 0.006*"liber" + 0.006*"parti" + 0.005*"say" + 0.005*"search"
Topic: 3 Word: 0.017*"countri" + 0.016*"hour" + 0.008*"podcast" + 0.008*"farm" + 0.007*"farmer" + 0.007*"price" + 0.006*"water" + 0.005*"council" + 0.005*"rural" + 0.005*"industri"
Topic: 4 Word: 0.017*"govern" + 0.008*"say" + 0.007*"minist" + 0.007*"michael" + 0.006*"state" + 0.006*"mark" + 0.006*"june" + 0.005*"stori" + 0.005*"andrew" + 0.005*"educ"
Topic: 5 Word: 0.015*"interview" + 0.009*"final" + 0.009*"australia" + 0.008*"leagu" + 0.008*"world" + 0.006*"test" +

In [29]:
processed_docs[4200]

['expand', 'shop', 'option', 'impact', 'chamber']

In [30]:
for index, score in sorted(lda_model[bow_corpus[4200]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.5167036056518555	 
Topic: 0.024*"chang" + 0.019*"miss" + 0.017*"tasmania" + 0.014*"farmer" + 0.014*"concern" + 0.013*"fall" + 0.013*"busi" + 0.013*"show" + 0.012*"price" + 0.012*"search"

Score: 0.1833157241344452	 
Topic: 0.038*"year" + 0.030*"queensland" + 0.022*"canberra" + 0.014*"investig" + 0.014*"children" + 0.014*"hous" + 0.011*"releas" + 0.011*"game" + 0.010*"storm" + 0.010*"life"

Score: 0.18328003585338593	 
Topic: 0.027*"elect" + 0.022*"sydney" + 0.017*"say" + 0.014*"labor" + 0.014*"record" + 0.013*"time" + 0.012*"call" + 0.012*"train" + 0.011*"polit" + 0.010*"budget"

Score: 0.01667519472539425	 
Topic: 0.035*"trump" + 0.021*"australia" + 0.020*"world" + 0.016*"final" + 0.016*"china" + 0.014*"rise" + 0.013*"island" + 0.012*"leagu" + 0.012*"take" + 0.011*"meet"

Score: 0.016670912504196167	 
Topic: 0.022*"warn" + 0.020*"market" + 0.018*"hospit" + 0.016*"health" + 0.014*"share" + 0.014*"break" + 0.014*"minist" + 0.013*"school" + 0.013*"student" + 0.013*"worker"

Sco

In [31]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.8199539184570312	 
Topic: 0.017*"countri" + 0.016*"hour" + 0.008*"podcast" + 0.008*"farm" + 0.007*"farmer" + 0.007*"price" + 0.006*"water" + 0.005*"council" + 0.005*"rural" + 0.005*"industri"

Score: 0.020010050386190414	 
Topic: 0.011*"coast" + 0.010*"south" + 0.009*"sport" + 0.009*"north" + 0.007*"east" + 0.007*"marriag" + 0.007*"west" + 0.007*"wednesday" + 0.007*"march" + 0.006*"festiv"

Score: 0.02000580169260502	 
Topic: 0.013*"queensland" + 0.012*"turnbul" + 0.012*"weather" + 0.010*"violenc" + 0.010*"victoria" + 0.008*"septemb" + 0.007*"perth" + 0.007*"domest" + 0.007*"inquest" + 0.007*"malcolm"

Score: 0.020005472004413605	 
Topic: 0.010*"grandstand" + 0.008*"christma" + 0.008*"octob" + 0.008*"mental" + 0.008*"asylum" + 0.007*"seeker" + 0.006*"health" + 0.006*"histori" + 0.006*"explain" + 0.005*"youth"

Score: 0.020004715770483017	 
Topic: 0.017*"charg" + 0.016*"polic" + 0.015*"trump" + 0.014*"murder" + 0.012*"court" + 0.011*"woman" + 0.010*"jail" + 0.009*"alleg" + 0.0

In [32]:
unseen_document = 'Trump strikes optimistic tone on China after weekend of contradictory signals '
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.3619116544723511	 Topic: 0.035*"trump" + 0.021*"australia" + 0.020*"world" + 0.016*"final" + 0.016*"china"
Score: 0.361844539642334	 Topic: 0.026*"kill" + 0.023*"home" + 0.020*"attack" + 0.019*"crash" + 0.017*"turnbul"
Score: 0.1267280876636505	 Topic: 0.038*"year" + 0.030*"queensland" + 0.022*"canberra" + 0.014*"investig" + 0.014*"children"
Score: 0.07945827394723892	 Topic: 0.024*"chang" + 0.019*"miss" + 0.017*"tasmania" + 0.014*"farmer" + 0.014*"concern"
Score: 0.01167741697281599	 Topic: 0.022*"warn" + 0.020*"market" + 0.018*"hospit" + 0.016*"health" + 0.014*"share"
Score: 0.011676453053951263	 Topic: 0.030*"govern" + 0.022*"south" + 0.021*"north" + 0.017*"countri" + 0.015*"rural"
Score: 0.01167590357363224	 Topic: 0.026*"adelaid" + 0.022*"test" + 0.017*"open" + 0.017*"tasmanian" + 0.017*"win"
Score: 0.01167590357363224	 Topic: 0.021*"die" + 0.019*"live" + 0.017*"donald" + 0.016*"coast" + 0.014*"gold"
Score: 0.01167590357363224	 Topic: 0.027*"elect" + 0.022*"sydney" + 0.01